Setup

In [2]:
#API connection and dependency install
import os
from flipside import Flipside
from dotenv import find_dotenv, load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Confirm and define dotenv path
dotenv_path = find_dotenv()
print(f"Loading .env from: {dotenv_path}")

#load env data from path
load_dotenv(dotenv_path, override=True)

#store API_KEY from dotenv file
import os
API_KEY = os.getenv("FLIP_API_KEY")
# print(f"API Key loaded: {API_KEY}")

#instantiate flipside client
flipside = Flipside(API_KEY, "https://api-v2.flipsidecrypto.xyz")

Loading .env from: c:\Users\savan\Desktop\portfolio_repo\arb_airdrop\.env
API Key loaded: 54ff2548-5f20-4730-9d97-3b40b935c69b


Wallet Retention

In [8]:
sql = f"""
WITH airdrop_recipients AS (
  SELECT 
    to_address AS user_address,
    block_timestamp AS claim_time,
    amount AS claimed_amount
  FROM arbitrum.core.ez_token_transfers
  WHERE 
    from_address = LOWER('0x67a24CE4321aB3aF51c2D0a4801c3E111D88C9d9')
    AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND amount > 0
),

token_balances AS (
  SELECT
    t.to_address AS user_address,
    SUM(t.amount) AS net_tokens_received
  FROM arbitrum.core.ez_token_transfers t
  INNER JOIN airdrop_recipients a
    ON t.to_address = a.user_address
  WHERE 
    t.contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND t.block_timestamp <= TIMESTAMPADD(DAY, 30, a.claim_time)
  GROUP BY t.to_address
)

SELECT
  COUNT(DISTINCT a.user_address) AS total_airdrop_users,
  COUNT(DISTINCT b.user_address) AS retained_users_after_30d,
  ROUND(COUNT(DISTINCT b.user_address) * 100.0 / COUNT(DISTINCT a.user_address), 2) AS retention_rate_pct
FROM airdrop_recipients a
LEFT JOIN token_balances b
  ON a.user_address = b.user_address;
"""
#query data and store locally
arb_retention = flipside.query(sql)

#create dataframe
columns = arb_retention.columns
rows = arb_retention.rows
arb_retention_df = pd.DataFrame(rows, columns=columns)

# Display head
display(arb_retention_df.head())



,total_airdrop_users,retained_users_after_30d,retention_rate_pct,__row_index
0,583138,583138,100,0


Arbitrum Bridge Rate

In [9]:
sql = f"""
WITH airdrop_recipients AS (
  SELECT 
    to_address AS user_address,
    block_timestamp AS claim_time
  FROM arbitrum.core.ez_token_transfers
  WHERE 
    from_address = LOWER('0x67a24CE4321aB3aF51c2D0a4801c3E111D88C9d9')
    AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND amount > 0
),

bridge_labels AS (
  SELECT address
  FROM crosschain.core.dim_labels
  WHERE blockchain = 'arbitrum'
    AND label_type = 'bridge'
),

bridging AS (
  SELECT DISTINCT
    t.from_address AS user_address
  FROM arbitrum.core.ez_token_transfers t
  INNER JOIN bridge_labels bl
    ON t.to_address = bl.address
  INNER JOIN airdrop_recipients a
    ON t.from_address = a.user_address
  WHERE t.block_timestamp <= TIMESTAMPADD(DAY, 30, a.claim_time)
)

SELECT
  COUNT(DISTINCT a.user_address) AS total_airdrop_users,
  COUNT(DISTINCT b.user_address) AS users_who_bridged,
  ROUND(COUNT(DISTINCT b.user_address) * 100.0 / COUNT(DISTINCT a.user_address), 2) AS bridge_rate_pct
FROM airdrop_recipients a
LEFT JOIN bridging b
  ON a.user_address = b.user_address;
"""
#query data and store locally
arb_bridge = flipside.query(sql)

#create dataframe
columns = arb_bridge.columns
rows = arb_bridge.rows
arb_bridge_df = pd.DataFrame(rows, columns=columns)

# Display head
display(arb_bridge_df.head())

,total_airdrop_users,users_who_bridged,bridge_rate_pct,__row_index
0,583138,76514,13.12,0


Arbitrum Swaps

In [12]:
sql = f"""
WITH airdrop_recipients AS (
  SELECT 
    to_address AS user_address,
    block_timestamp AS claim_time
  FROM arbitrum.core.ez_token_transfers
  WHERE 
    from_address = LOWER('0x67a24CE4321aB3aF51c2D0a4801c3E111D88C9d9')
    AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND amount > 0
),

swaps AS (
  SELECT DISTINCT
    s.origin_from_address AS user_address
  FROM arbitrum.defi.ez_dex_swaps s
  INNER JOIN airdrop_recipients a
    ON s.origin_from_address = a.user_address
  WHERE 
    s.block_timestamp <= TIMESTAMPADD(DAY, 30, a.claim_time)
    AND (s.symbol_in = 'ARB' OR s.symbol_out = 'ARB')
)

SELECT
  COUNT(DISTINCT a.user_address) AS total_airdrop_users,
  COUNT(DISTINCT s.user_address) AS users_who_swapped,
  ROUND(COUNT(DISTINCT s.user_address) * 100.0 / COUNT(DISTINCT a.user_address), 2) AS swap_rate_pct
FROM airdrop_recipients a
LEFT JOIN swaps s
  ON a.user_address = s.user_address;
"""
#query data and store locally
arb_swap = flipside.query(sql)

#create dataframe
columns = arb_swap.columns
rows = arb_swap.rows
arb_swap_df = pd.DataFrame(rows, columns=columns)

# Display head
display(arb_swap_df.head())

,total_airdrop_users,users_who_swapped,swap_rate_pct,__row_index
0,583138,355109,60.9,0


Arbitrum Post Drop Engagement

In [13]:
sql = f"""
WITH airdrop_recipients AS (
  SELECT 
    to_address AS user_address,
    block_timestamp AS claim_time
  FROM arbitrum.core.ez_token_transfers
  WHERE 
    from_address = LOWER('0x67a24CE4321aB3aF51c2D0a4801c3E111D88C9d9')
    AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND amount > 0
),

post_claim_tx AS (
  SELECT DISTINCT 
    t.to_address AS user_address
  FROM arbitrum.core.ez_token_transfers t
  INNER JOIN airdrop_recipients a
    ON t.to_address = a.user_address
  WHERE t.block_timestamp <= TIMESTAMPADD(DAY, 30, a.claim_time)
)

SELECT
  COUNT(DISTINCT a.user_address) AS total_airdrop_users,
  COUNT(DISTINCT p.user_address) AS engaged_users,
  ROUND(COUNT(DISTINCT p.user_address) * 100.0 / COUNT(DISTINCT a.user_address), 2) AS engagement_rate_pct
FROM airdrop_recipients a
LEFT JOIN post_claim_tx p
  ON a.user_address = p.user_address;
"""
#query data and store locally
arb_drop_engagment = flipside.query(sql)

#create dataframe
columns = arb_drop_engagment.columns
rows = arb_drop_engagment.rows
arb_drop_engagment_df = pd.DataFrame(rows, columns=columns)

# Display head
display(arb_drop_engagment_df.head())

,total_airdrop_users,engaged_users,engagement_rate_pct,__row_index
0,583138,583138,100,0


Arbitrum Drop Engagement Score Distribution

In [15]:
sql = f"""
WITH airdrop_recipients AS (
  SELECT 
    to_address AS user_address,
    block_timestamp AS claim_time
  FROM arbitrum.core.ez_token_transfers
  WHERE 
    from_address = LOWER('0x67a24CE4321aB3aF51c2D0a4801c3E111D88C9d9')
    AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND amount > 0
),

-- 1. Holding Behavior after 30 days
token_balance_30d AS (
  SELECT
    t.to_address AS user_address,
    SUM(t.amount) AS net_tokens_received
  FROM arbitrum.core.ez_token_transfers t
  INNER JOIN airdrop_recipients a
    ON t.to_address = a.user_address
  WHERE 
    t.contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    AND t.block_timestamp >= TIMESTAMPADD(DAY, 30, a.claim_time)
  GROUP BY t.to_address
),

-- 2. DEX Swap Activity
dex_swaps AS (
  SELECT DISTINCT
    s.origin_from_address AS user_address
  FROM arbitrum.defi.ez_dex_swaps s
  INNER JOIN airdrop_recipients a
    ON s.origin_from_address = a.user_address
  WHERE 
    s.block_timestamp >= TIMESTAMPADD(DAY, 1, a.claim_time)
    AND (s.symbol_in = 'ARB' OR s.symbol_out = 'ARB')
),

-- 3. Bridging Activity
bridge_labels AS (
  SELECT address
  FROM crosschain.core.dim_labels
  WHERE blockchain = 'arbitrum'
    AND label_type = 'bridge'
),

bridging AS (
  SELECT DISTINCT
    t.from_address AS user_address
  FROM arbitrum.core.ez_token_transfers t
  INNER JOIN bridge_labels bl
    ON t.to_address = bl.address
  INNER JOIN airdrop_recipients a
    ON t.from_address = a.user_address
  WHERE t.block_timestamp >= TIMESTAMPADD(DAY, 1, a.claim_time)
),

-- Combine everything to compute score
engagement_scores AS (
  SELECT
    a.user_address,
    
    -- Score Components:
    CASE WHEN tb.user_address IS NOT NULL THEN 1 ELSE 0 END AS holding_score,
    CASE WHEN ds.user_address IS NOT NULL THEN 1 ELSE 0 END AS dex_swap_score,
    CASE WHEN br.user_address IS NOT NULL THEN 1 ELSE 0 END AS bridging_score,
    -- Total Score:
    (CASE WHEN tb.user_address IS NOT NULL THEN 1 ELSE 0 END
     + CASE WHEN ds.user_address IS NOT NULL THEN 1 ELSE 0 END
     + CASE WHEN br.user_address IS NOT NULL THEN 1 ELSE 0 END) AS engagement_score
  
  FROM airdrop_recipients a
  LEFT JOIN token_balance_30d tb
    ON a.user_address = tb.user_address
  LEFT JOIN dex_swaps ds
    ON a.user_address = ds.user_address
  LEFT JOIN bridging br
    ON a.user_address = br.user_address
)

-- Aggregation
SELECT
  engagement_score,
  COUNT(*) AS num_users
FROM engagement_scores
GROUP BY engagement_score
ORDER BY engagement_score DESC;
    
"""
#query data and store locally
arb_engagment_score = flipside.query(sql)

#create dataframe
columns = arb_engagment_score.columns
rows = arb_engagment_score.rows
arb_engagment_score_df = pd.DataFrame(rows, columns=columns)

# Display head
display(arb_engagment_score_df.head())

,engagement_score,num_users,__row_index
0,3,52936,0
1,2,157654,1
2,1,227384,2
3,0,145164,3
